In [19]:
%config IPCompleter.greedy=True
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import Image, clear_output
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import cv2
import os
import platform

In [20]:
# Create results folder one time
if not os.path.exists("results"):
    os.mkdir("results")
    
# Include the .dll file in lib folder
# Needed for creating and exporting the video using cv2
libFolder = os.path.abspath('lib/' + platform.system())
os.environ['PATH'] = libFolder + ";" + os.environ['PATH']

In [21]:
# Load COVID-19 Data to cDate dataframe 
cData = pd.read_csv('COVID-19 Data/covid_daily.csv')

In [22]:
# Main GUI - Create Figures' Buttons
title_label    = widgets.Label(value='Choose Visualization Mode')
bubble_btn     = widgets.Button(description='Bubble')
map_btn        = widgets.Button(description='Map')
bar_btn        = widgets.Button(description='Bar')
btns_container = widgets.VBox([title_label, widgets.HBox([bubble_btn, map_btn, bar_btn])])
outputFigure   = widgets.Output()


def createGraphs():
    """
    Creates the graph figure for each mode (Bubble, Map and Bar).
    Each graph is created using plotly-express (high-level API)
    
    """
    
    print('Creating Bubble Graph...')
    bubbleFigure = px.scatter(cData, x="Deaths",
                                     y="Recovered",
                                     size="Confirmed",
                                     title='COVID-19 Bubble Graph',
                                     color="Confirmed",
                                     color_continuous_scale='Viridis',
                                     text="Country",
                                     animation_frame="Date",
                                     animation_group="Country",
                                     hover_name="Country",
                                     log_x=True,
                                     log_y=True,
                                     size_max=200)
    print('Bubble Graph Created Successfully!')
    
    print('Creating Map Graph...')
    mapFigure = px.choropleth(cData, locations='Country',
                                      locationmode='country names',
                                      title='COVID-19 Map Graph',
                                      color='Confirmed',
                                      color_continuous_scale='Viridis',
                                      labels={'Country':'Confirmed'},
                                      hover_name='Country',
                                      animation_frame="Date",
                                      animation_group="Country")
    print('Map Graph Created Successfully!')

    print('Creating Bar Graph...')
    # Remove the countires which have 0 confirmed cases in each day
    updatedData = cData[cData['Confirmed'] != 0]
    barFigure = px.bar(updatedData, x="Country",
                                    y="Confirmed",
                                    title='COVID-19 Bar Graph',
                                    color_continuous_scale='Viridis',
                                    animation_frame="Date",
                                    animation_group="Country")
    
    # For updating the bars to be sorted descending
    barFigure.update_layout(xaxis={'categoryorder': 'total descending'})
    print('Bar Graph Created Successfully!')
    
    return {'Bubble': bubbleFigure, 'Map': mapFigure, 'Bar': barFigure}

def displayBubbleGraph(_):
    with outputFigure:
        clear_output()
        figures['Bubble'].show()

def displayMapGraph(_):
    with outputFigure:
        clear_output()
        figures['Map'].show()
        
def displayBarGraph(_):
    with outputFigure:
        clear_output()
        figures['Bar'].show()
        
figures = createGraphs()

Creating Bubble Graph...
Bubble Graph Created Successfully!
Creating Map Graph...
Map Graph Created Successfully!
Creating Bar Graph...
Bar Graph Created Successfully!


In [23]:
# Buttons Handler 
bubble_btn.on_click(displayBubbleGraph)
map_btn.on_click(displayMapGraph)
bar_btn.on_click(displayBarGraph)
widgets.VBox([btns_container, outputFigure])

In [24]:
def ExportGraphToMP4(graphName: str, width: int, height: int, FPS: int, Seconds: int):
    video_dimensions = (width, height)
    fps = FPS
    seconds = Seconds
    fourcc = cv2.VideoWriter_fourcc(*'avc1')    
    video = cv2.VideoWriter(f"results/{graphName}Graph.mp4", fourcc, fps, video_dimensions)
    figCounter = 0
    
    print(f'Exporting {graphName} Graph to mp4 video, please wait..')
    # The Main Loop Over The Video To Draw Each Frame
    for i in range(0, seconds*fps):
        print(f'Current Frame Of Data {figCounter}')
        if i % 8 == 0:
            print(f'Get New Frame...')
            if figCounter < len(figures[graphName].frames):
                fig = go.Figure(figures[graphName].frames[figCounter].data, figures[graphName].layout)
                img_bytes = fig.to_image(format="png")
                stream = BytesIO(img_bytes)
                image = Image.open(stream).convert("RGBA")
                stream.close()
            else:
                break

        # Draw frame.
        video.write(cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR))

        # Start/End of the frame
        # In case of 60 fps -> each figure will be drawn 15 times
        if (i+1) % fps//4 == 0:
            figCounter += 1
        
    video.release()
    print(f'Exporting finished Successfully! in results/{graphName}Graph.mp4')
        


In [ ]:
# videodims = (700, 500)
# fps = 60
# seconds = 60
# fourcc = cv2.VideoWriter_fourcc(*'avc1')    
# video = cv2.VideoWriter(graphName + "Graph.mp4", fourcc, fps, videodims)
# figCounter = 0

# # The Main Loop Over The Video To Draw Each Frame
# for i in range(0, seconds*fps):
#     print(f'Current Figure Frame {figCounter}')
#     if i % 8 == 0:
#         print(f'Get New Figure Frame')
#         if figCounter < len(figures['Bubble'].frames):
#             fig = go.Figure(figures['Bubble'].frames[figCounter].data, figures['Bubble'].layout)
#             img_bytes = fig.to_image(format="png")
#             stream = BytesIO(img_bytes)
#             image = Image.open(stream).convert("RGBA")
#             stream.close()
#         else:
#             break
        
#     # Draw frame.
#     video.write(cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR))
    
#     # Start/End of the frame
#     if (i+1) % 15 == 0:
#         figCounter += 1
        
# video.release()

In [ ]:
# img_bytes = figures['Bubble'].to_image(format="png")
# img_bytes

In [ ]:
# figures['Bubble'].write_image("images/fig1.png")

In [ ]:
# fig1 = go.Figure(figures['Bubble'].frames[0].data, figures['Bubble'].layout)
# fig1.show()

In [ ]:
# img_bytes = fig1.to_image(format="png")
# img_bytes

In [ ]:
# stream = BytesIO(img_bytes)
# # stream = BytesIO(img_bytes.encode())
# image = Image.open(stream).convert("RGBA")
# stream.close()
# image.show()

In [ ]:
# imagesList = []
# for frame in figures['Bubble'].frames:
#     fig = go.Figure(frame.data, figures['Bubble'].layout)
#     img_bytes = fig.to_image(format="png")
#     stream = BytesIO(img_bytes)
#     image = Image.open(stream).convert("RGBA")
#     stream.close()
#     imagesList.append(image)
    
#     videodims = (700,100)
#     fourcc = cv2.VideoWriter_fourcc(*'avc1')    
#     video = cv2.VideoWriter("test.mp4",fourcc, 60,videodims)
#     img = Image.new('RGB', videodims, color = 'darkred')
#     #draw stuff that goes on every frame here
#     for i in range(0,60*60):
#         imtemp = img.copy()
#         # draw frame specific stuff here.
#         video.write(cv2.cvtColor(np.array(imtemp), cv2.COLOR_RGB2BGR))
#     video.release()
    
# imagesList[0].save('results/BubbleGraph.mp4',
#                    save_all=True, append_images=imagesList[1:], optimize=True, duration=100, loop=1)

In [ ]:
# img_bytes = figures['Bubble'].to_image(format="png")
# img_bytes

In [8]:
# figures['Bubble'].write_image("images/fig1.png")

In [9]:
# fig1 = go.Figure(figures['Bubble'].frames[0].data, figures['Bubble'].layout)
# fig1.show()

In [10]:
# img_bytes = fig1.to_image(format="png")
# img_bytes

In [11]:
# stream = BytesIO(img_bytes)
# # stream = BytesIO(img_bytes.encode())
# image = Image.open(stream).convert("RGBA")
# stream.close()
# image.show()

Current Figure Frame 0
Get New Figure Frame
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Get New Figure Frame
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 0
Current Figure Frame 1
Get New Figure Frame
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Get New Figure Frame
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 1
Current Figure Frame 2
Get New Figure Frame
Current Figure Frame 2
Current Figure Frame 2
Current Figure Frame 2
Current Figure Frame 2
Current Figure Frame 2
Current Figure Frame 

Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Get New Figure Frame
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 20
Current Figure Frame 21
Get New Figure Frame
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Get New Figure Frame
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 21
Current Figure Frame 22
Get New Figure Frame
Current Figure Frame 22
Current Figure Frame 22
Current Figure Frame 22
Current Figure Frame 22
Current Figure Frame 22
Current Figure Frame 22
Current Figure Frame 22
Curr

Current Figure Frame 39
Current Figure Frame 40
Get New Figure Frame
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Get New Figure Frame
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 40
Current Figure Frame 41
Get New Figure Frame
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Get New Figure Frame
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 41
Current Figure Frame 42
Get New Figure Frame
Current Figure Frame 42
Current Figure Frame 42
Current Figure Frame 42
Current

Current Figure Frame 59
Current Figure Frame 59
Current Figure Frame 59
Current Figure Frame 59
Current Figure Frame 59
Current Figure Frame 60
Get New Figure Frame
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Get New Figure Frame
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 60
Current Figure Frame 61
Get New Figure Frame
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Get New Figure Frame
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 61
Current Figure Frame 62
Get 

Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Get New Figure Frame
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 79
Current Figure Frame 80
Get New Figure Frame
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Get New Figure Frame
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 80
Current Figure Frame 81
Get New Figure Frame
Current Figure Frame 81
Current Figure Frame 81
Current Figure Frame 81
Current Figure Frame 81
Current Figure Frame 81
Current Figure Frame 81
Curr

Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 98
Current Figure Frame 99
Get New Figure Frame
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Get New Figure Frame
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 99
Current Figure Frame 100
Get New Figure Frame
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Get New Figure Frame
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure Frame 100
Current Figure

In [13]:
# imagesList = []
# for frame in figures['Bubble'].frames:
#     fig = go.Figure(frame.data, figures['Bubble'].layout)
#     img_bytes = fig.to_image(format="png")
#     stream = BytesIO(img_bytes)
#     image = Image.open(stream).convert("RGBA")
#     stream.close()
#     imagesList.append(image)
    
#     videodims = (700,100)
#     fourcc = cv2.VideoWriter_fourcc(*'avc1')    
#     video = cv2.VideoWriter("test.mp4",fourcc, 60,videodims)
#     img = Image.new('RGB', videodims, color = 'darkred')
#     #draw stuff that goes on every frame here
#     for i in range(0,60*60):
#         imtemp = img.copy()
#         # draw frame specific stuff here.
#         video.write(cv2.cvtColor(np.array(imtemp), cv2.COLOR_RGB2BGR))
#     video.release()
    
# imagesList[0].save('results/BubbleGraph.mp4',
#                    save_all=True, append_images=imagesList[1:], optimize=True, duration=100, loop=1)